<a href="https://colab.research.google.com/github/fedraameneiro/DataScientistFinalProject/blob/main/Proyecto_final_Ciencia_de_Datos_FedraAmeneiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto diabetes



#**Abstract**

La diabetes diabetes mellitus es una enfermedad crónica que se presenta cuando el páncreas no secreta suficiente insulina o cuando el organismo no utiliza eficazmente la insulina que produce. La insulina es una hormona que regula la concentración de glucosa en la sangre, es decir, la glucemia. Un efecto común de la diabetes no controlada es la hiperglucemia (es decir, la glucemia elevada) que, con el tiempo daña gravemente muchos órganos y sistemas del organismo, sobre todo los nervios y los vasos sanguíneos.

En 2014, el 8,5% de los mayores de 18 años padecían diabetes. En 2019, esta afección fue la causa directa de 1,5 millones de defunciones y, de todos los fallecidos por diabetes, el 48% tenía menos de 70 años. Además, otras 460 000 personas fallecieron a causa de la nefropatía diabética, y la hiperglucemia ocasiona alrededor del 20% de las defunciones por causa cardiovascular (1).

Entre 2000 y 2019, las tasas de mortalidad por diabetes normalizadas por edades aumentaron en un 3%. En los países ingresos medianos o bajos, la tasa de mortalidad por diabetes aumentó en un 13%.

En cambio, entre 2000 y 2019, la probabilidad de fallecer entre los 30 y los 70 años de edad por alguna de las cuatro principales enfermedades no transmisibles (enfermedades cardiovasculares, cáncer, enfermedades respiratorias crónicas o diabetes) se redujo en un 22% a escala mundial.
https://www.who.int/es/news-room/fact-sheets/detail/diabetes

Realizamos el análisis de 100.000 muestras de pacientes hasta los 80 años y analizamos la probabilidad de sufrir diabetes según las variables de hipertensión, problemas cardiovasculares, histórico fumando, índice corporal, nivel de HbA1c y glucosa en sangre.

Queremos saber cuáles son las variables que más nos afectan a la hora de sufrir esta enfermedad.


1.   Qué variables son las preponderantes en el riesgo de tener diabetes?
2.   Qué probabilidad tiene un paciente de sufrir diabetes teniendo en cuenta las variables menos fuertes?

Finalmente analizaremos un algoritmo de Machine learning que mejor nos prediga esta condición.




**Glosario**

*   gender  :               Género del paciente. Hombre o mujer.
*   age     :               Edad del paciente en el momento de la muestra.
*   hypertension    :       Si el paciente presenta hipertensión como patología es 1.
*   heart_disease  :        Si el paciente presenta patologías cardiovasculares, es 1.
*   smoking_history    :    Antecedentes en la relación con el tabaco.
*   bmi             :       Indice de masa corporal.
*   HbA1c_level     :       Niveles de HbA1c.
*   blood_glucose_level :    Niveles de glucosa en sangre.
*   diabetes          :     Si el diagnóstico final es con diabetes es 1.













In [ ]:
#librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import plotly as pl
import plotly.express as px


In [ ]:
#obtener el archivo csv de drive

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link = 'https://drive.google.com/file/d/18ymyIaVZ8FK7WhyGltLrJbfx2nyvzjtK/view?usp=drive_link'

# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('diabetes.csv')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
df=pd.read_csv('diabetes.csv', sep=';')
df.head()

Tenemos 4 variables categoricas y la variable objetivo. Y 3 variables numericas.

In [ ]:
df.dtypes

In [ ]:
df.count()

Antes de nada voy a chequear aquellas  variables numericas que aparecen como objetos y los voy a modificar. bmi y HbA1c_level deberían ser enteros. Les voy a cambiar la coma por un punto.

In [ ]:
df.bmi=df.bmi.str.replace(',','.')
df.HbA1c_level=df.HbA1c_level.str.replace(',','.')
df.bmi=df["bmi"].astype(float)
df.HbA1c_level=df["HbA1c_level"].astype(float)
df.dtypes

In [ ]:
df.describe().T

In [ ]:
df_diabetes=df[df.diabetes==1]
df_nodiabetes=df[df.diabetes==0]
df_diabetes.head()

#Ingeniería de factores
Feacture creation y Feacture Cleaning

In [ ]:
df.info()

In [ ]:
df.age.value_counts()

In [ ]:
df.isnull().sum()/df.shape[0]

No hay nulos

In [ ]:
df.isna().sum()

No hay NaN

Heatmap entre todas las variables numericas medidas

In [ ]:
plt.figure(dpi = 120,figsize= (20,8))
mask = np.triu(np.ones_like(df.corr(),dtype = bool))
sns.heatmap(df.corr(),mask = mask, fmt = ".2f",annot=True,lw=1,cmap="Blues")
plt.yticks(rotation = 0)
plt.xticks(rotation = 90)
plt.title('Correlation Heatmap')
plt.show()

Relación entre nuestras variables categoricas

In [ ]:
pd.crosstab(df.gender, df.smoking_history)

Eliminamos aquellas muestras con genero other.

In [ ]:
df['gender'].unique()

In [ ]:
df=df[df.gender!='Other']

In [ ]:
pd.crosstab(df.gender, df.smoking_history)

#Analisis exploratorio de los datos. EDA

#Analisis Univariado


Vemos la cantidad de diabéticos de nuestro estudio, son relativamente pocos en relación al estudio total, pero suficientes. 8500 de 100000 pacientes, 8.5%.

In [ ]:
#cantidad total de registros
len(df)

In [ ]:
#cantidad de diabeticos en total
df.diabetes.sum()

Revisamos la relación entre ser diabetico y no en nuestras muestras

In [ ]:
numero_de_diabeticos = len(df_diabetes)
numero_de_no_diabeticos= len(df_nodiabetes)

In [ ]:
medidas=[numero_de_diabeticos,numero_de_no_diabeticos]
labels =['Diabeticos','No diabeticos']
colores = ["#EE6055","#60D394"]
#plt.pie(medidas, labels=nombres, autopct="%0.1f %%", colors=colores)

plt.pie(medidas, labels=labels, autopct="%.1f%%",explode=[0.05]*2, pctdistance=0.5, colors=colores)
plt.title("Distribucion de barrios", fontsize=14)


In [ ]:
df_nodiabetes.pivot_table(values='blood_glucose_level', index='gender')

In [ ]:
df.pivot_table(values='blood_glucose_level', index='gender')
df_diabetes.pivot_table(values='blood_glucose_level', index='gender')

In [ ]:
df_diabetes.pivot_table(values='age', index='smoking_history', columns=['gender','blood_glucose_level'])

#Análisis categórico
Empezamos analizando las variables categóricas para los diabeticos, separamos las muestras de pacientes que sufren diabetes y que no y revisamos las variables que más afectan.


*   Género
*   Historial como fumador
*   Hipertensión
*   Enfermedades Cardiovasculares


Relación de diabeticos masculinos y femeninos. No vemos especial relación en ser diabético y el género.

In [ ]:
plt.hist(df_nodiabetes['gender'], bins=10, alpha=0.5, label='No diabeticos')  # alpha controla la transparencia
plt.hist(df_diabetes['gender'], bins=10, alpha=0.5, label='Diabeticos')

# Configuración de la leyenda y etiquetas
plt.legend(loc='upper right')
plt.xlabel('Valor')
plt.ylabel('Genero')
plt.title('Género')

# Mostrar el gráfico
plt.show()

Relación entre el consumo de tabaco y la enfermedad. De la misma forma, no hay relación entre el consumo de tabaco y la diabetes.

In [ ]:

plt.hist(df_nodiabetes['smoking_history'], bins=10, alpha=0.5, label='No diabeticos')  # alpha controla la transparencia
plt.hist(df_diabetes['smoking_history'], bins=10, alpha=0.5, label='Diabeticos')

# Configuración de la leyenda y etiquetas
plt.legend(loc='upper right')
plt.xlabel('Valor')
plt.ylabel('Tabaco')
plt.title('Smoking History')

# Mostrar el gráfico
plt.show()

Relación entre el consumo de tabaco y la hipertensión, no hay mayor ponderancia de diabéticos que sufran hipertensión en relación a los pacientes que no resultaron con la enfermedad.

In [ ]:
plt.hist(df_nodiabetes['hypertension'], bins=10, alpha=0.5, label='No diabeticos')  # alpha controla la transparencia
plt.hist(df_diabetes['hypertension'], bins=10, alpha=0.5, label='Diabeticos')

# Configuración de la leyenda y etiquetas
plt.legend(loc='upper right')
plt.xlabel('Valor')
plt.ylabel('hypertension')
plt.title('Sufre Hipertensión')

# Mostrar el gráfico
plt.show()

Y finalmente, tampoco vemos relación entre sufrir cardiopatías y sufrir diabetes.

In [ ]:
plt.hist(df_nodiabetes['heart_disease'], bins=10, alpha=0.5, label='No diabeticos')  # alpha controla la transparencia
plt.hist(df_diabetes['heart_disease'], bins=10, alpha=0.5, label='Diabeticos')

# Configuración de la leyenda y etiquetas
plt.legend(loc='upper right')
plt.xlabel('Valor')
plt.ylabel('heart_disease')
plt.title('Sufre alguna Cardiopatía')

# Mostrar el gráfico
plt.show()

#Analisis numérico
A continuación vamos a analizar las variables numéricas y determinar si alguna nos afecta en la probabilidad de sufrir diabetes.


*   Edad
*   Bmi, índice de masa corporal
*   Niveles de HbA1C
*   Niveles de glucosa en sangre



Relación entre ser diabético y la edad. Mas edad, más probabilidad. Y esta relación no se ve afectada con el género, como se vió antes.

In [ ]:
plt.hist(df_nodiabetes['age'], bins=30, alpha=0.5, label='No diabeticos')  # alpha controla la transparencia
plt.hist(df_diabetes['age'], bins=30, alpha=0.5, label='Diabeticos')

# Configuración de la leyenda y etiquetas
plt.legend(loc='upper right')
plt.xlabel('Valor')
plt.ylabel('age')
plt.title('Edad')

# Mostrar el gráfico
plt.show()

In [ ]:
sns.histplot(df_diabetes,x='age',hue='gender')

Relación entre ser diabético y el indice de masa corporal BMI. Podemos ver que a medida que aumenta la edad, y a mayor indice de masa corporal, la probabilidad de diabetes aumenta.

In [ ]:
px.scatter(df,x='age',y='bmi',color='diabetes')

In [ ]:
px.scatter(df,x='HbA1c_level',y='blood_glucose_level',color='diabetes')

Relacion entre ser diabetico y los niveles de HbA1c. En este caso podemos ver que a partir de 6,5 hacia arriba, los valores indican directamente una relación con la diabetes.

In [ ]:
plt.hist(df_nodiabetes['HbA1c_level'], bins=10, alpha=0.5, label='No diabeticos')  # alpha controla la transparencia
plt.hist(df_diabetes['HbA1c_level'], bins=10, alpha=0.5, label='Diabeticos')

# Configuración de la leyenda y etiquetas
plt.legend(loc='upper right')
plt.xlabel('Valor')
plt.ylabel('HbA1c_level')
plt.title('HbA1c_level')

# Mostrar el gráfico
plt.show()

Relacion entre el ser diabetico y el nivel de glucosa en sangre. Vemos que, siendo 150 niveles normales y habituales en los dos grupos, a mayores valores de glucosa en sangre le corresponden directamente casos de diabetes, mientras que no existen casos en los valores más bajos. Por lo tanto hay una relación directa entre ser diabético y el nivel de glucosa en sangre.

In [ ]:
plt.hist(df_nodiabetes['blood_glucose_level'], bins=10, alpha=0.5, label='No diabeticos')  # alpha controla la transparencia
plt.hist(df_diabetes['blood_glucose_level'], bins=10, alpha=0.5, label='Diabeticos')

# Configuración de la leyenda y etiquetas
plt.legend(loc='upper right')
plt.xlabel('Valor')
plt.ylabel('blood_glucose_level')
plt.title('Nivel de glucosa en sangre')

# Mostrar el gráfico
plt.show()

#Conclusiones:
No hay relación directa de la diabetes con variables categóricas como el género, ser fumador, antecedentes de problemas cardiovasculares o hipertensión.

Vemos relación de la edad con la diabetes, a más edad mayor probabilidad de diabetes. Pero sobre todo, se vió una relación entre la edad y el índice corporal.

La mayor correlación se vio entre la diabetes y las variables numericas está en HbA1c_level y blood glucose level.

# Algoritmo Machine Learning
Queremos predecir si un paciente es diabético o no.
Vamos a seleccionar qué algoritmo funciona mejor para la previsión de que un paciente sea diabético según ciertos parámetros.



*   Regresión logística
*   KNN
*   SVM
*   Arboles de decisión
*   Redes neuronales



In [ ]:
df.head()

Vemos la correlación entre variables numéricas y confirmamos que las mejores variables que se relacionan con la diabetes son el nivel HbA1c_level y el nivel de glucosa en sangre.

In [ ]:
X=df[[ 'HbA1c_level','blood_glucose_level']]
y=df['diabetes']

In [ ]:
X

**Regresion Logistica**

Empezamos construyendo un modelo de regresión logística con las variables que más fuerza hemos visto que tienen en la predicción de la condición de diabetes.

In [ ]:
# Importar biblioteca
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
X=df[[ 'HbA1c_level','blood_glucose_level']]
y=df['diabetes']

# Dividir el conjunto de datos en conjuntos de entrenamiento y testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:



# Inicializar el modelo de regresión logística
model = LogisticRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predictions = model.predict(X_test)

# Calcular la precisión del modelo
accuracy = np.mean(predictions == y_test)
print(f"Precisión del modelo: {accuracy * 100:.2f}%")



Es una buena opción de clasificación como diabético, no diabético.
Este caso es un modelo con pocas variables, sin embargo podemos conseguir una cantidad de muestras de entrenamiento bastante amplia.

Comparamos este modelo con las cuatro variables que hemos visto que pueden predecir la condición: Edad, PMI, glucose_blood y HbA1c_level.


In [ ]:
X=df[['age', 'bmi','HbA1c_level','blood_glucose_level']]
y=df['diabetes']

# Dividir el conjunto de datos en conjuntos de entrenamiento y testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

# Inicializar el modelo de regresión logística
model = LogisticRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predictions = model.predict(X_test)

# Calcular la precisión del modelo
accuracyRegresionLineal = np.mean(predictions == y_test)
print(f"Precisión del modelo con 'age', 'bmi','HbA1c_level','blood_glucose_level': {accuracyRegresionLineal * 100:.2f}%")



**KNN**

vamos a probar con el modelo KNN.

In [ ]:
# Importar las bibliotecas necesarias
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler



In [ ]:
X=df[['age', 'bmi','HbA1c_level','blood_glucose_level']]
y=df['diabetes']

# Dividir el conjunto de datos en conjuntos de entrenamiento y testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Inicializar el clasificador KNN con un valor de k (número de vecinos) deseado
n_neighbors=5 # Puedes ajustar el valor de k según tu necesidad
knn = KNeighborsClassifier(n_neighbors)

# Entrenar el modelo con el conjunto de entrenamiento
knn.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = knn.predict(X_test)

# Calcular la precisión del modelo
accuracyKNN = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracyKNN * 100:.2f}%")

print('Precision del K-NN en entrenamiento: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Precision del  K-NN en testing: {:.2f}'
     .format(knn.score(X_test, y_test)))

Confirmemos la precisión viendo la Confusión Matrix y el Reporte sobre el conjunto de test, que nos detalla los aciertos y fallos:

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

En nuestra matriz podemos ver que el valor de f1 para el valor con diabetes es de 0.80, un valor bastante bueno.

La precision de nuestro modelo es excelente.

KNN da mejores valores en su modelo

Vamos a revisar la grafica con los resultados de los valores, apoyandonos que tenemos dos opciones posibles y podemos observarlo en 2 dimensiones.

In [ ]:
X

**SVM**

In [ ]:
modelSVC.fit(X_train,y_train)
y_pred2=modelSVC.predict(X_test)
from sklearn.metrics import accuracy_score
accuracySVM =accuracy_score(y_pred2,y_test)

**Arbol de decisión**

In [ ]:
#Cargamos la librería para utilizar el modelo arbol de decisión
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X=df[['age', 'bmi','HbA1c_level','blood_glucose_level']]
y=df['diabetes']

# Dividir el conjunto de datos en conjuntos de entrenamiento y testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crea el modelo de árbol de decisión
modelo_arbol = DecisionTreeClassifier()

# Entrena el modelo con los datos de entrenamiento
modelo_arbol.fit(X_train, y_train)

# Realiza predicciones en el conjunto de prueba
predicciones = modelo_arbol.predict(X_test)

# Evalúa la precisión del modelo
accuracyArbolesDecisión = accuracy_score(y_test, predicciones)
print(f"Precisión del modelo Arboles de decisión: {accuracyArbolesDecisión * 100:.2f}%")

In [ ]:
# Puedes usar el modelo entrenado para hacer predicciones en nuevos datos


#diabetes=0
nuevo_paciente = {
    'age': 76,
    'bmi': 27,
    'HbA1c_level': 5,
    'blood_glucose_level': 160
}

# Convertir los datos del nuevo paciente en un DataFrame
nuevo_df = pd.DataFrame([nuevo_paciente])

# Realizar la predicción utilizando el modelo de árbol de decisión entrenado
prediccion_diabetes = modelo_arbol.predict(nuevo_df)

# Mostrar el resultado de la predicción
if prediccion_diabetes[0] == 1:
    print("El paciente tiene diabetes.")
else:
    print("El paciente no tiene diabetes.")

**Redes Neuronales**


In [ ]:
# Importa las bibliotecas necesarias
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
cX=df[['age', 'bmi','HbA1c_level','blood_glucose_level']]
y=df['diabetes']

# Normaliza las características (es importante para las redes neuronales)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir el conjunto de datos en conjuntos de entrenamiento y testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Construir el modelo de red neuronal
modelo = Sequential()
modelo.add(Dense(32, input_dim=4, activation='relu'))
modelo.add(Dense(16, activation='relu'))
modelo.add(Dense(1, activation='sigmoid'))



In [ ]:

# Compila el modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Entrena el modelo con los datos de entrenamiento. epoch=1000
modelo.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evalúa el modelo con los datos de prueba
precisionRedesNeuronales = modelo.evaluate(X_test, y_test) [1]
print(f'Precisión del modelo: {precisionRedesNeuronales}')


**Resultados**

Cuál es el modelo de predicción que mejor se ajusta a nuestro problema?

In [ ]:
print(f"Precisión del modelo de Regresión logística: {accuracyRegresionLineal * 100:.2f}%")
print(f"Precisión del modelo de KNN: {accuracyKNN * 100:.2f}%")
print(f"Precisión del modelo SVM: {accuracySVM* 100:.2f}%")
print(f"Precisión del modelo de Arboles de decisión: {accuracyArbolesDecisión * 100:.2f}%")
print(f"Precisión del modelo Redes Neuronales: {precisionRedesNeuronales* 100:.2f}%")

El mejor modelo de aprendizaje para predecir Diabetes en un paciente según las variables que tenemos, es el modelo de redes neuronales.
Vamos a aplicar este modelo, entrenarlo y predecir nuestro resultado a partir de valores cualquiera.

Voy a empezar metiendo datos que ya tenemos y revisamos si efectivamente predice de la manera correcta:

In [ ]:
valor_glucosa=140
valor_bmi=26.1
valor_edad=21
valorHbA1c=5.8
#diabetes=0

# Puedes usar el modelo entrenado para hacer predicciones en nuevos datos
paciente= np.array([[valor_edad, valor_bmi, valorHbA1c, valor_glucosa]])
nuevos_datos_scaled = scaler.transform(paciente)

prediccion_diabetes_paciente = modelo.predict(nuevos_datos_scaled)

# Convierte las probabilidades en predicciones binarias usando un umbral (por ejemplo, 0.5)
prediccion_binaria =  (prediccion_diabetes_paciente >= 0.5).astype(int)
# Mapea las predicciones binarias a etiquetas más descriptivas
resultado = "Diabetes" if prediccion_binaria[0] == 1 else "No Diabetes"
print(f'Predicción para paciente : {resultado}')
prediccion_diabetes_paciente

In [ ]:
valor_glucosa=160
valor_bmi=25.91
valor_edad=73
valorHbA1c=9
#diabetes=1

#predice la condición del paciente a partir de las cuatro variables medidas
paciente= np.array([[valor_edad, valor_bmi, valorHbA1c, valor_glucosa]])
nuevos_datos_scaled = scaler.transform(paciente) #es necesario hacerlo
prediccion_diabetes_paciente = modelo.predict_on_batch(nuevos_datos_scaled)

# Convierte las probabilidades en predicciones binarias usando un umbral (por ejemplo, 0.5)
prediccion_binaria = (prediccion_diabetes_paciente >= 0.5).astype(int)

# Mapea las predicciones binarias a etiquetas más descriptivas
resultado = "Diabetes" if prediccion_binaria[0] == 1 else "No Diabetes"
print(f'Predicción para paciente : {resultado}')
prediccion_diabetes_paciente

In [ ]:
valor_glucosa=90
valor_bmi=36.38
valor_edad=40
valorHbA1c=6
#diabetes=0
# Puedes usar el modelo entrenado para hacer predicciones en nuevos datos
paciente= np.array([[valor_edad, valor_bmi, valorHbA1c, valor_glucosa]])
nuevos_datos_scaled = scaler.transform(paciente)
prediccion_diabetes_paciente = modelo.predict(nuevos_datos_scaled)
# Convierte las probabilidades en predicciones binarias usando un umbral (por ejemplo, 0.5)
prediccion_binaria = (prediccion_diabetes_paciente >= 0.5).astype(int)
# Mapea las predicciones binarias a etiquetas más descriptivas
resultado = "Diabetes" if prediccion_binaria[0] == 1 else "No Diabetes"
print(f'Predicción para paciente : {resultado}')

In [ ]:
valor_glucosa=159
valor_bmi=37.16
valor_edad=50
valorHbA1c=9
#diabetes=1
# Puedes usar el modelo entrenado para hacer predicciones en nuevos datos
paciente= np.array([[valor_edad, valor_bmi, valorHbA1c, valor_glucosa]])
nuevos_datos_scaled = scaler.transform(paciente)
prediccion_diabetes_paciente = modelo.predict(nuevos_datos_scaled)
# Convierte las probabilidades en predicciones binarias usando un umbral (por ejemplo, 0.5)
prediccion_binaria = (prediccion_diabetes_paciente >= 0.5).astype(int)
# Mapea las predicciones binarias a etiquetas más descriptivas
resultado = "Diabetes" if prediccion_binaria[0] == 1 else "No Diabetes"
print(f'Predicción para paciente : {resultado}')

#Entrenamiento y mejora del algoritmo

#**Cross validation**

Evaluamos los distintos modelos a través de cross validation

In [ ]:
X=df[['age', 'bmi','HbA1c_level','blood_glucose_level']]
y=df['diabetes']


In [ ]:
# Normaliza las características (es importante para las redes neuronales)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir el conjunto de datos en conjuntos de entrenamiento y testing
x_train, y_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=7)

In [ ]:
def evaluate_model(models):
    """
    Takes a list of models and returns chart of cross validation scores using mean accuracy
    """

    # Cross validate model with Kfold stratified cross val
    kfold = StratifiedKFold(n_splits = 10)

    result = []
    for model in models :
        result.append(cross_val_score(estimator = model, X = x_train, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=4))

    cv_means = []
    cv_std = []
    for cv_result in result:
        cv_means.append(cv_result.mean())
        cv_std.append(cv_result.std())

    result_df = pd.DataFrame({
        "CrossValMeans":cv_means,
        "CrossValerrors": cv_std,
        "Models":[
            "LogisticRegression",
            "DecisionTreeClassifier",
            "SVC",
            "RandomForestClassifier",
            "KNeighborsClassifier"
        ]
    })

    # Generate chart
    bar = sns.barplot(x = "CrossValMeans", y = "Models", data = result_df, orient = "h")
    bar.set_xlabel("Mean Accuracy")
    bar.set_title("Cross validation scores")
    return result_df

In [ ]:
# Modeling Libraries
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, train_test_split

In [ ]:
# Modeling step Test differents algorithms
random_state = 30
models = [
    LogisticRegression(random_state = random_state, solver='liblinear'),
    DecisionTreeClassifier(random_state = random_state),
    SVC(random_state = random_state),
    RandomForestClassifier(random_state = random_state),
    KNeighborsClassifier()
]
evaluate_model(models)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('Models').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

Elegimos el mejor valor de k

In [ ]:
k_range = range(1,20)
scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    scores.append(knn.score(X_test, y_test))
plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, scores)
plt.xticks([0,5,10,15,20])

De esta grafica podemos interpretar que a partir del valor K=3 la precision es la misma, y perfecta.

Vamos a usar el algoritmo con nuevos pacientes y valores

In [ ]:
#print(clf.predict([[5, 1.0]]))

#PCA- Principal Component Analysis

Chequeamos las variables independientes que mejor funcionan en nuestro modelo mediante la técnica de PCA.

Scaleamos los datos para más adelante comparar las variables

In [ ]:
df.head()

eliminamos las variables que ya usamos en nuestro modelo (blood_glucose_level y HbA1c_level y convertimos nuestras variables categoricas en numericas.

In [ ]:
df2=df.drop('HbA1c_level',axis=1)
df2=df2.drop('blood_glucose_level',axis=1)
df2=pd.get_dummies(df2)
df2.head()

In [ ]:
#cargamos librerias PCA
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [ ]:
std = StandardScaler()
X_std=std.fit_transform(df2)
df_std = pd.DataFrame(X_std,columns=df2.columns)
df_std.head()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca_df = PCA()
x_transformado=pca_df.fit_transform(df_std)
x_transformado

In [ ]:
df_nuevo=pd.DataFrame(x_transformado,columns=['pc'+str(x) for x in range(1,14)])

In [ ]:
df_nuevo

In [ ]:
pca_df.explained_variance_ratio_

In [ ]:
plt.figure(figsize=(6,5))

plt.plot(range(1,14), pca_df.explained_variance_ratio_, '-o', label='Componente individual')
plt.plot(range(1,14), np.cumsum(pca_df.explained_variance_ratio_), '-s', label='Acumulado')

plt.ylabel('Porcentaje de Varianza Explicada'); plt.xlabel('Componentes Principales')
plt.ylim(0,1.05); plt.xticks(range(13))
plt.legend(loc=0);

Esta grafica nos representa que no tengo ninguna otra variable que me aporte nada más que las dos variables que usamos en nuestro modelo KNN.

#SHAP

qué variables tienen más influencia en las predicciones de las técnicas de machine learning.

In [ ]:
!pip install shap
import shap

In [ ]:
X=df[['age', 'bmi','HbA1c_level','blood_glucose_level']]
y=df['diabetes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
import lightgbm as lgb

gbm = lgb.LGBMRegressor()
model=gbm.fit(X_train, y_train)

In [ ]:
#aca usamos SHAP
explainer = shap.Explainer(model)


In [ ]:
explainer.expected_value

In [ ]:
shap_values = explainer(X_test)
shap_values

In [ ]:
X_test.shape

In [ ]:
# tengo 6k miro el 15
shap.initjs()# en colab, tiene que ir en la misma celda
shap.plots.force( shap_values[0],figsize=(15, 3))


In [ ]:
shap.plots.bar(shap_values)

In [ ]:
# localmente, para una prediccion, veo la importancia de las features usando los valores shap
# miro el primero 0
shap.plots.waterfall(shap_values[30])

In [ ]:
X=df[['age', 'bmi','HbA1c_level','blood_glucose_level']]
y=df['diabetes']

# Normaliza las características (es importante para las redes neuronales)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir el conjunto de datos en conjuntos de entrenamiento y testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Construir el modelo de red neuronal
modelo = Sequential()
modelo.add(Dense(32, input_dim=4, activation='relu'))
modelo.add(Dense(16, activation='relu'))
modelo.add(Dense(1, activation='sigmoid'))



In [ ]:

# Compila el modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Entrena el modelo con los datos de entrenamiento. epoch=1000
modelo.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)



In [ ]:
#aca usamos SHAP
explainer = shap.Explainer(model)


In [ ]:
explainer.expected_value

In [ ]:
shap_values = explainer(X_test)
shap_values

In [ ]:
X_test.shape

In [ ]:
# tengo 6k miro el 15
shap.initjs()# en colab, tiene que ir en la misma celda
shap.plots.force( shap_values[0],figsize=(15, 3))


In [ ]:
shap.plots.bar(shap_values)